In [45]:
import tweepy
import configparser

In [46]:
config = configparser.ConfigParser()
config.read('config.ini')

api_key = config['twitter']['api_key']
api_key_secret = config['twitter']['api_key_secret']

access_token = config['twitter']['access_token']
access_token_secret = config['twitter']['access_token_secret']
print(api_key)
print(api_key_secret)
print(access_token)
print(access_token_secret)

D36zKRt2oqq38TjnRQZc4zPwh
x2Gke0pktMnOdWliytf4fQqMc2vwAHIyUDKz7TkSBzY0pVaWuc
1517394779260473344-FInfqjNiYmkXqPHzifd5wQ2bM0lZhI
P2tmPTGOa5kH2gRsUOXCDRrfpFNwjtK9PWmPmDkNrUpPJ


In [47]:
def lookup_tweets(tweet_IDs, api):
    full_tweets = []
    tweet_count = len(tweet_IDs)
    try:
        for i in range(int((tweet_count / 100) + 1)):
            # Catch the last group if it is less than 100 tweets
            end_loc = min((i + 1) * 100, tweet_count)
            full_tweets.extend(
                client.get_tweet(id=tweet_IDs[i * 100:end_loc])
            )
        return full_tweets
    except tweepy.errors.TweepyException:
        print('Something went wrong, quitting...')

In [51]:
auth = tweepy.OAuthHandler(api_key,api_key_secret)
auth.set_access_token(access_token, access_token_secret)

client = tweepy.Client(auth)

text = client.get_tweet(1250219116993974272)

Unauthorized: 401 Unauthorized

In [52]:
results = lookup_tweets([1250219300389974016,1250219116993974272], api)

for tweet in results:
    if tweet:
        print(tweet.text)

Something went wrong, quitting...


TypeError: 'NoneType' object is not iterable